In [1]:
import numpy as np
import re, os
import Function

In [2]:
parent_dir = os.path.abspath(os.path.dirname(os.getcwd()))

In [3]:
# Create folder
def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    # Check if the path exists
    isExists=os.path.exists(path)
    if not isExists:
        # Create the directory if it doesn't exist
        os.makedirs(path)
    else:
        # Do not create directory if it exists
        pass

In [4]:
def read_pre_res(filepath):
    with open(filepath) as f:
        records = f.readlines()
        f.close()
        res = []
        for i in records:
            if 'fold ' in i:
                fold_res = []
                res.append(fold_res)
                continue
            fold_res.append(i.rstrip().split('\t') if i.rstrip() != '' else None)
    return np.array(res).astype(np.float64)

In [5]:
def calculate_cutoff(data, sp_value=0.65):
    cutoffs = []
    for fold in data:
        neg = []
        for value in fold:
            if value[0] == 0.0:
                neg.append(list(value))
        negative = np.array(neg)
        all_n = len(negative)
        tn = int(sp_value*all_n)
        fp = all_n - tn
        data = negative[np.argsort(-negative[:,1])]
        cutoff = data[:,1][fp-1]
        cutoffs.append(cutoff)
    return cutoffs

In [6]:
filepathCv = os.path.join(parent_dir, r'Results\RSCNN_ZScale\zscale_pre_cv.txt')
filepathInd = os.path.join(parent_dir, r'Results\RSCNN_ZScale\zscale_pre_ind.txt')

In [7]:
Code_Name = 'Fixd_sp'

In [10]:
predict_cv_res = read_pre_res(filepathCv)
cutoffs_cv = calculate_cutoff(predict_cv_res, sp_value=0.65)
predict_ind_res = read_pre_res(filepathInd)
cutoffs_ind = calculate_cutoff(predict_ind_res, sp_value=0.65)
out = os.path.join(parent_dir, 'Results')
subout = os.path.join(out, Code_Name)
mkdir(subout)
save = os.path.join(subout, Code_Name.lower())

In [11]:
cv_metrics = Function.fixed_sp_calculate_metrics_list(predict_cv_res, cutoffs_cv, label_column=0, score_column=1, po_label=1)
Function.save_prediction_metrics_list(cv_metrics, save + '_metrics_cv.txt')
ind_metrics = Function.fixed_sp_calculate_metrics_list(predict_ind_res, cutoffs_ind, label_column=0, score_column=1, po_label=1)
Function.save_prediction_metrics_list(ind_metrics, save + '_metrics_ind.txt')